<a href="https://colab.research.google.com/github/lee-thien-tuyen/Digit-Recognizer-/blob/main/digit_recognizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# download data file from github
! git clone https://github.com/lee-thien-tuyen/Digit-Recognizer-.git 

In [ ]:
import pandas as pd
import numpy as np 
import tensorflow as tf 
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 

In [ ]:
train = pd.read_csv("Digit-Recognizer-/train.csv")
test = pd.read_csv("Digit-Recognizer-/test.csv")

train.head(8)

In [ ]:
train.info()

In [ ]:
train.describe()

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
y_train = train['label'].astype('float32')
x_train = train.drop(['label'],axis = 1).astype('int32')

x_test = test.astype('float32')

print("x_train have shape:",x_train.shape)
print("y_train have shape:",y_train.shape)
print("x_test have shape:",x_test.shape)

In [ ]:
x_train = x_train.values.reshape(-1,28,28)
x_test = x_test.values.reshape(-1,28,28)
print("x_train after reshape:",x_train.shape)
print("x_test after reshape:",x_test.shape)

In [ ]:
# one hot coding y
y_train = tf.keras.utils.to_categorical(y_train,10)
print("y_train shape:", y_train.shape)

print(y_train[0:5,:])

In [ ]:
#split data
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train,y_train,test_size = 0.2,random_state = 42)
print("x_train shape:",x_train.shape)
print("y_train shape:", y_train.shape)

In [ ]:
plt.figure(figsize=(12, 2), tight_layout=True, facecolor="white")

for idx, digit in enumerate(x_train[:10]):
    plt.subplot(1, 10, idx + 1) 
    plt.imshow(digit, cmap="binary")
    plt.axis("off")
    
plt.show()

In [ ]:
from scipy.ndimage import rotate, shift


plt.figure(figsize=(12, 5), tight_layout=True, facecolor="white")
max_dx, max_dy, max_angle = 4, 4, 20

# Example operations with Scipy.
for row, digit in enumerate(x_train[:3]):
    for col in range(10):
        dx = np.random.randint(-max_dx, max_dx)
        dy = np.random.randint(-max_dy, max_dy)
        angle = np.random.randint(-max_angle, max_angle)

        shifted = shift(digit, (dx, dy))
        rotated = rotate(shifted, angle, order=1, reshape=False)

        plt.subplot(3, 10, (col+1) + row * 10)
        plt.imshow(rotated, cmap="binary")
        plt.axis("off")

plt.show()

In [ ]:
def digits_augmentation(X, y, max_dx=4, max_dy=4, max_angle=20, repeats=10):
    X_augmented = [digit for digit in X]
    y_augmented = [label for label in y]

    for digit, label in zip(X, y):
        for _ in range(repeats):
            dx = np.random.randint(-max_dx, max_dx)
            dy = np.random.randint(-max_dy, max_dy)
            angle = np.random.randint(-max_angle, max_angle)

            shifted = shift(digit, (dx, dy))
            rotated = rotate(shifted, angle, order=1, reshape=False)

            X_augmented.append(rotated)
            y_augmented.append(label)

    return np.array(X_augmented), np.array(y_augmented)

In [ ]:
np.random.seed(42)
X_augmented, y_augmented = digits_augmentation(x_train, y_train)

#shuff data
shuffled_ids = np.random.permutation(len(X_augmented))

X_augmented = X_augmented[shuffled_ids]
y_augmented = y_augmented[shuffled_ids] 

X_augmented = np.expand_dims(X_augmented, axis=-1) / 255.0
x_val = np.expand_dims(x_val, axis=-1) / 255.0
x_test = np.expand_dims(x_test, axis=-1) / 255.0

print("X_augmented shape:", X_augmented.shape)
print("x_val shape:", x_val.shape)
print("x_test shape:", x_test.shape)
print("y_augmented shape:",y_augmented.shape)

In [ ]:
#define callback function to stop our training when the target accuracy is reached
class mycallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self,epoch,logs={}):
    if (logs.get('accuracy') is not None and logs.get('accuracy') >0.999):
      print("\n Reached 99.9% accuracy so cancelling training!!!")
      self.model.stop_training = True 


In [ ]:
# Define function model NNs
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32,3,activation = 'relu',input_shape = (28,28,1),kernel_initializer="he_normal"),
    tf.keras.layers.Conv2D(32,3,activation = 'relu',kernel_initializer="he_normal"),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64,3,activation = 'relu',padding = 'same'),
    tf.keras.layers.Conv2D(64,3,activation = 'relu',padding = 'same'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64,3,activation = 'relu',padding = 'same'),
    tf.keras.layers.Conv2D(64,3,activation = 'relu',padding = 'same'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2),strides=(2,2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256,activation = 'relu'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(256,activation = 'relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10,activation = 'softmax')
])
model.summary()

In [ ]:
# Compile and fit model
optimizer = tf.keras.optimizers.Adam(
                                learning_rate= 0.001,
                                beta_1 = 0.9,
                                beta_2 = 0.999,
                                epsilon = 1e-07,
                                name = 'Adam')
callbacks = mycallback()

model.compile(optimizer = optimizer,
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])
model.fit(X_augmented,y_augmented,batch_size =128,epochs = 20,validation_data = (x_val,y_val),callbacks=[callbacks])


In [ ]:
results = model.predict(x_test)

#select the index with the maximum probaility 
results = np.argmax(results,axis = 1)

results = pd.Series(results,name = "Label")

In [ ]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

submission.to_csv("submission.csv",index = False)